In [ ]:
import pandas as pd
import numpy as np
import re
import glob
from pathlib import Path
from datetime import datetime
from datetime import timedelta

In [ ]:
ts = datetime.now().strftime('%Y%m%d')

In [ ]:
df = pd.concat(pd.read_csv(f) for f in glob.glob(f"output/clean_csv/*.csv"))
df

In [ ]:
df.info()

In [ ]:
def track_sold (df, sku_col = 'link_product', date_col = 'update_date'):
    df[date_col]= pd.to_datetime(df[date_col], format='%Y%m%d', errors='coerce')
    max_date = df[date_col].max()

    last_seen = (df.groupby(sku_col)[date_col].max().rename('sold_date'))
    sold_dates = last_seen[last_seen < max_date] + timedelta(days=1)
    df['sold_date'] = df[sku_col].map(sold_dates)

    return df

In [ ]:
def track_import (df, sku_col = 'link_product', date_col = 'update_date'):
    df[date_col]= pd.to_datetime(df[date_col], format='%Y%m%d', errors='coerce')
    first_seen = (df.groupby(sku_col)[date_col].min().rename('import_date'))
    df['import_date'] = df[sku_col].map(first_seen)

    return df

In [ ]:
df = track_sold(df)
df = track_import(df)
df['import_date'] = df['import_date'].replace('2025-11-05',None)


df

In [ ]:
output = Path ('output')
output.mkdir(parents=True, exist_ok=True)
filename= f'{ts}_data.csv'
file_path = output / filename
df.to_csv(file_path, index=False, encoding="utf-8-sig")